In [1]:
#Como cargar un fichero de Excel, necesitaremos la libreria openpyxl
import pandas as pd
fichero = r"C:\Users\formacio\Downloads\VentasGroucery_Meses.xlsx"
hojaAgosto = "Ventas Agosto 2014"
Agosto2014 = pd.read_excel(fichero,sheet_name=hojaAgosto)
hojaSeptiembre = "Ventas Septiembre 2014"
Septiembre2014 = pd.read_excel(fichero,sheet_name=hojaSeptiembre)
hojaOctubre = "Ventas Octubre 2014"
Octubre2014 = pd.read_excel(fichero,sheet_name=hojaOctubre)
#Anexar para tener un solo DataFrame
trimestre2014 = pd.concat([Agosto2014,Septiembre2014,Octubre2014],ignore_index=True)
#print(trimestre2014.head(5))
#print(trimestre2014.tail(5))
#Ahora vamos a calcular el Amount = Unit Price * Quantity
trimestre2014['Amount'] = trimestre2014['Unit Price'] * trimestre2014['Quantity']
#print(trimestre2014.info())
#Ahora vamos a hacer una agrupación, toda agrupación necesita 2 cosas, dimensiones y metricas
#dimensiones = todo aquello por lo que queremos medir (Category, Product, Ship City, Customer)
#metrica = la medición de la información (Suma de Amount, Promedio de Unit Price, La cantidad
#de pedidos) se hace con funciones de resumen o estadistica.
#la suma de Amount por Pais
trimestre2014.groupby(['Ship Country']).Amount.sum()
#El promedio de precio unidad por ciudad, si el nombre tiene espacios en blanco, no se puede
#llamar con el punto, si no con el corchete
trimestre2014.groupby(['Ship City'])['Unit Price'].mean()
#El numero de pedidos por categoria
trimestre2014.groupby(['Category'])['Order ID'].count()
#Vamos a agrupar por Pais y Ciudad
trimestre2014.groupby(['Ship Country','Ship City'])['Amount'].sum()
#trimestre2014.groupby(['Ship Country','Ship City']).Amount.sum()

#La suma por categoria, voy a guardar esta agrupación en un objeto
Category_Amount = trimestre2014.groupby(['Category'])['Amount'].sum()
Category_Amount   #Este objeto no es un dataframe
Category_Amount = pd.DataFrame(Category_Amount, columns=['Amount'])
#Para generar el indice numerico a la vez que Category se convierte en una columna
Category_Amount = Category_Amount.reset_index()
#Si voy a usar estos datos en Excel, Power BI u otro programa, debera cambiar punto
#por coma
Category_Amount['Amount'] = Category_Amount['Amount'].astype(str).str.replace(".",",")
#Como guardar este o cualquier otro dataframe en un fichero csv
archivo_csv = r"C:\temp\Category_Amount.csv"
Category_Amount.to_csv(archivo_csv,sep=";",index=True,decimal=",")
print("Guardado con éxito",archivo_csv)
#Como guardar en un fichero de excel
archivo_excel = r"C:\temp\Category_Amount.xlsx"
Category_Amount.to_excel(archivo_excel, sheet_name="Agrupación", header=True, index=False)
print("Guardado con éxito",archivo_excel)

Guardado con éxito C:\temp\Category_Amount.csv
Guardado con éxito C:\temp\Category_Amount.xlsx


In [ ]:
import pandas as pd

pedidos = pd.read_excel(r"C:\Users\formacio\Downloads\Pedidos.xlsx", sheet_name="Pedidos")
print(pedidos.info())

detallepedidos = pd.read_excel(r"C:\Users\formacio\Downloads\Detalles Pedidos.xlsx",
                               sheet_name="Detalles Pedidos")
print(detallepedidos.info())
#Vamos a aprender a combinar 2 dataframes, concat pondremos filas una debajo de las otras
#con merge pondremos columna unas al lado de las otras, para hacer merge necesitamos un campo
#clave en cada dataframe, con on seria el nombre de el campo con el que se vinculan
#si son diferentes nombres haremos lo siguiente, para hacer el merge el nombre del campo
#no tiene porque llamarse igual, el tipo de dato debe ser el mismo, numero, texto, booleano, fecha
#el tipo de combinación es inner join (solo mostrara el que coincida en los 2 dataframes), por
#defecto es un inner
merge_pedidos = pd.merge(pedidos,detallepedidos, on="Id. de pedido")
merge_pedidos
#merge_pedidos = pd.merge(pedidos,detallepedidos, left_on="Id. de pedido", 
#                         right_on="Id. de pedido", how="inner")

PRACTICA 32:
CARGAREMOS Y COMBINAREMOS LOS SIGUIENTES FICHEROS DE EXCEL, PEDIDOS, DETALLES PEDIDO, PRODUCTO
CATEGORIA, REALIZAREMOS LAS TRANSFORMACIONES NECESARIAS PARA GUARDAR EN UN FICHERO DE EXCEL
EN NUESTRO PC, UN CONJUNTO DE DATOS QUE CONTENGA LA SUMA DE IMPORTE (PRECIO UNIDAD * CANTIDAD)
, EL NUMERO DE PEDIDOS, EL PROMEDIO DE CANTIDAD AGRUPADO POR CATEGORIA.
NO SE FACILITAN MAS DATOS, COMO EN LA REAL LIFE.

OS DEJO 9-10 MINUTOS Y RESOLVEMOS ESTA PRACTICA 32

In [ ]:
pedidos = pd.read_excel(r"C:\Users\formacio\Downloads\Pedidos.xlsx", sheet_name="Pedidos")
detallepedidos = pd.read_excel(r"C:\Users\formacio\Downloads\Detalles Pedidos.xlsx",
                               sheet_name="Detalles Pedidos")
productos = pd.read_excel(r"C:\Users\formacio\Downloads\Productos.xlsx", sheet_name="Productos")
categorias = pd.read_excel(r"C:\Users\formacio\Downloads\Categorías.xlsx", sheet_name="Categorías")
#Juntar los primeros 2 dataframe pedidos + detalles de pedidos
merge_pedidos = pd.merge(pedidos,detallepedidos, on="Id. de pedido")
#Juntar Productos con Categorías
merge_pro_cate = pd.merge(productos, categorias, left_on="Categoría", right_on="Nombre de categoría")
#Tener el total
merge_total = pd.merge(merge_pedidos,merge_pro_cate, left_on="Producto", right_on="Nombre de producto")
#merge_total.info()
#Vamos a tener el DataFrame con solo 4 columnas, usaremos lo siguiente
merge_total = merge_total[['Id. de pedido','Precio por unidad','Cantidad','Nombre de categoría']]
merge_total['Importe'] = merge_total['Precio por unidad']*merge_total['Cantidad']
merge_total
#La suma de importe
Importe_Categoría = merge_total.groupby(['Nombre de categoría'])['Importe'].sum()
Importe_Categoría = pd.DataFrame(Importe_Categoría, columns=['Importe'])
Importe_Categoría = Importe_Categoría.reset_index()
Importe_Categoría
#La cantidad de pedidos por Categoría
Pedidos_Categoría = merge_total.groupby(['Nombre de categoría'])['Id. de pedido'].count()
Pedidos_Categoría = pd.DataFrame(Pedidos_Categoría, columns=['Id. de pedido'])
Pedidos_Categoría = Pedidos_Categoría.reset_index()
Pedidos_Categoría
#El promedio de cantidad por Categoría
Promedio_Categoría = merge_total.groupby(['Nombre de categoría'])['Cantidad'].mean()
Promedio_Categoría = pd.DataFrame(Promedio_Categoría, columns=['Cantidad'])
Promedio_Categoría = Promedio_Categoría.reset_index()
Promedio_Categoría
#Haremos un merge de los 3 dataframes agrupados
resultado1 = pd.merge(Importe_Categoría,Pedidos_Categoría,on="Nombre de categoría")
resultado2 = pd.merge(resultado1,Promedio_Categoría, on="Nombre de categoría")
resultado2 = resultado2.rename(columns={'Id. de pedido':'Num. pedidos',
                                        'Cantidad':'Promedio Cantidad'})
resultado2

,Nombre de categoría,Importe,Num. pedidos,Promedio Cantidad
0,Bebidas,286526.95,404,23.594059
1,Carnes,178188.80,173,24.271676
2,Condimentos,113694.75,216,24.527778
3,Frutas/Verduras,105268.60,136,21.985294
4,Granos/Cereales,100726.80,196,23.275510
5,Lácteos,251330.50,366,24.997268
6,Pescado/Marisco,141623.09,330,23.275758
7,Repostería,177099.10,334,23.670659
